In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import time as dt_time
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.5)
pd.set_option('display.float_format', '{:.2f}'.format)


def night_adjust(dt: datetime) -> datetime:
    if dt.time() > dt_time(20, 0):
        return (dt - timedelta(1))
    else:
        return dt


def strip_digt(symbol: str) -> str:
    res = ""
    for char in symbol:
        if not char.isdigit():
            res += char
        else:
            break
    return res


def trade_stats(df: pd.DataFrame) -> pd.DataFrame:
    pos = df[df['pnl_by_trade'] > 0]
    neg = df[df['pnl_by_trade'] <= 0]
    profit_mean = pos['pnl_by_trade'].mean()
    loss_mean = neg['pnl_by_trade'].mean()
    
    d = {
        'profit_num': len(pos),
        'loss_num': len(neg),
        'all_num': len(df),
        'accuracy_rate': len(pos) / len(df),
        'profit_mean': profit_mean,
        'loss_mean': loss_mean,
        'ratio': profit_mean / abs(loss_mean)
    }
    return pd.DataFrame([d])
    

In [ ]:
# load data
df = pd.read_csv('trade_clean_demo2.csv',
                 usecols=[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 17])
new_cols = [
    'account', 'contract', 'direction', 'price', 'volume', 'turnover',
    'offset', 'commission', 'pnl_by_date', 'pnl_by_trade', 'time', 'date'
]
df.columns = new_cols

# change data type
df[['account', 'date']] = df[['account', 'date']].astype('str')
df['underlying'] = df['contract'].map(strip_digt)
col_lst = ['turnover', 'pnl_by_date', 'pnl_by_trade']
for col in col_lst:
    df[col] = df[col].map(lambda x: float(x.replace(',', '')))

# add datetime and adjust night trade order
df['dt'] = df['date'] + df['time']
df['dt'] = df['dt'].map(lambda x: datetime.strptime(x, '%Y%m%d%H:%M:%S'))
df['dt'] = df['dt'].map(night_adjust)
df.sort_values(by='dt', inplace=True)
df.to_csv('trade_cleaned.csv', encoding='utf-8-sig')

In [ ]:
df3 = df.groupby(['contract', 'dt','direction', 'offset', 'price']).agg(sum)
df3.head(100)
# df3.to_csv('cleaned.csv')

In [ ]:
df_dt = df.groupby(['dt','contract', 'direction', 'offset', 'price']).agg(sum)
df_dt.to_csv('cleaned_trade_dt.csv')
df_dt.head(100)

In [ ]:
df_under = df.groupby(['dt','underlying', 'contract', 'direction', 'offset', 'price']).agg(sum)
df_under.to_csv('cleaned_trade_under.csv')
df_under.head()

In [ ]:
df_res = df[df.offset != '开仓']
df_res.head()
df_stats = df_res.groupby(['underlying', 'contract']).apply(trade_stats)
df_stats.to_csv('trade_stats.csv')

# df_stats = df_res.groupby('underlying').apply(trade_stats)
# df_stats.to_csv('trade_stats_underlying.csv', index=False)
df_stats